#### The Dataset used in this project is available on kaggle: Intel Image classification

https://www.kaggle.com/puneet6060/intel-image-classification

In [94]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision

import pathlib

#### Let us check the device used

In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [96]:
#### Let us transform

#### Let us transform for Data processing

In [97]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [98]:
pwd

'/home/joram/Videos/From Here/Intel Classifier/archive'

#### Let us Download Dataloader

In [99]:
# Paths fr training and testing
train_path = ('/home/joram/Videos/From Here/Intel Classifier/archive/seg_train/seg_train')
test_path  = ('/home/joram/Videos/From Here/Intel Classifier/archive/seg_test/seg_test')


In [108]:
train_path_bu = ('/home/joram/Videos/From Here/Intel Classifier/archive/seg_train/seg_train/buildings')
print('buildings: ',len(train_path_bu))
print('----------------------------------')
train_path_fo= ('/home/joram/Videos/From Here/Intel Classifier/archive/seg_train/seg_train/forest')
print('forest: ',len(train_path_fo))
print('----------------------------------')

buildings:  83
----------------------------------
forest:  80
----------------------------------


In [103]:
# image, label = train_path_vu[0]
# plt.imshow(image, cmap='gray')
# print('Label:', label)

In [ ]:
train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path, transform=transformer), batch_size=256, shuffle=True)

test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path, transform=transformer), batch_size=256, shuffle=True)

#### Let us see all the classes

In [81]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [82]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


#### Let us define CNN

In [83]:

class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [84]:
model=ConvNet(num_classes=6).to(device)

#### Let us set the optimizer and loss function

In [85]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [86]:
num_epochs = 10

#### Let us calculating the size of training and testing images

In [87]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [88]:
print(train_count, test_count)

14034 3000


In [89]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(8.0279) Train Accuracy: 0.5488812882998433 Test Accuracy: 0.6726666666666666
Epoch: 1 Train Loss: tensor(1.2280) Train Accuracy: 0.7271626051018953 Test Accuracy: 0.7073333333333334
Epoch: 2 Train Loss: tensor(0.8483) Train Accuracy: 0.7965654838249965 Test Accuracy: 0.6546666666666666
Epoch: 3 Train Loss: tensor(0.7381) Train Accuracy: 0.8252102037907938 Test Accuracy: 0.601
Epoch: 4 Train Loss: tensor(0.7215) Train Accuracy: 0.8361122987031495 Test Accuracy: 0.73
Epoch: 5 Train Loss: tensor(0.5318) Train Accuracy: 0.8766566908935443 Test Accuracy: 0.661
Epoch: 6 Train Loss: tensor(0.2840) Train Accuracy: 0.9214051588998148 Test Accuracy: 0.7126666666666667
Epoch: 7 Train Loss: tensor(0.2010) Train Accuracy: 0.9451332478267066 Test Accuracy: 0.725
Epoch: 8 Train Loss: tensor(0.2920) Train Accuracy: 0.9320934872452615 Test Accuracy: 0.723
Epoch: 9 Train Loss: tensor(0.2346) Train Accuracy: 0.9440644149921619 Test Accuracy: 0.7546666666666667
